Gather necessary pre-requisites:

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
from nilearn import plotting, datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nibabel as nb
import os, re
from itertools import zip_longest

200228-16:39:38,725 nipype.utils INFO:
	 Running nipype version 1.5.0-dev (latest: 1.4.2)


In [2]:
# define useful functions
# define show-slices function
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
# add suptitle via plt.suptitle
        
# check size of items and delete them when above a certain threshold
from sys import getsizeof # function is not fully indicative -> look for better option
def check_itemsize(items):
    for i in items:
        # return size of object in bytes
        print("size of item", i, "is:", getsizeof(i))
        if getsizeof(i) > 10000: # think about dynamic coding for this
            del i
            print("deleted, object exceeded set threshold.")
        else:
            print("retained, object is sufficiently small.")

Get files

In [3]:
# collect all spmT images [contrasts 1-4: dep>con, con>dep, male>female, young>old]
# specify relevant directories
VBM_dir = "/VBM/"
DARTEL_template_dir = VBM_dir + "DARTEL_template/"
MReg_dir = VBM_dir + "SPM_MultiReg/estimate_MReg/"
con_dir = MReg_dir + "estimate_Contrats/"

# get spmT images
spmT_img = sorted([i for i in os.listdir(con_dir) if re.match(r'(^spmT.*)', i)])

# get directories of FDR thresholded images
FDR_dirs = [os.path.join(con_dir, "FDR_spmT%s" %(contrast_index)) for contrast_index in range(1, len(spmT_img)+1)]

# get the spmT images after FDR thresholding (show only surviving clusters)
spmT_img_FDR = [os.path.join(subdir, item) for subdir in FDR_dirs for item in os.listdir(subdir) 
                if re.match(r'(^spmT.*\d{4}_thr.nii)', item)]

# define contrast names in order of creation (MReg model)
con_names = ["dep>con", "con>dep", "male>female", "young>old"]

In [4]:
print(spmT_img_FDR)

['/VBM/SPM_MultiReg/estimate_MReg/estimate_Contrats/FDR_spmT1/spmT_0001_thr.nii', '/VBM/SPM_MultiReg/estimate_MReg/estimate_Contrats/FDR_spmT2/spmT_0002_thr.nii', '/VBM/SPM_MultiReg/estimate_MReg/estimate_Contrats/FDR_spmT3/spmT_0003_thr.nii', '/VBM/SPM_MultiReg/estimate_MReg/estimate_Contrats/FDR_spmT4/spmT_0004_thr.nii']


Plot VBM results

In [5]:
# get surf_mesh background for 3D plotting
#fsaverage = datasets.fetch_surf_fsaverage()

# prep image
# flexible type error -> look at nb object and access datatype
#malefemale_nb = nb.load(spmT_img_FDR[2])
# get func data, change datatype from nan to float32
#malefemale_fdata = malefemale_nb.dataobj[:].astype(np.float32)
# save back to nifti, using Identity matrix as affine transformation
#malefemale_fdata_img = nb.Nifti1Image(malefemale_fdata, np.eye(4))
#malefemale_fdata_img.to_filename(os.path.join(FDR_dirs[2], "spmT3.nii.gz"))

#stat_img_surf_3D = plotting.plot_surf_stat_map(stat_map=os.path.join(FDR_dirs[2], "spmT3.nii.gz"), title=con_names[2], 
#                                       surf_mesh=fsaverage['pial_left'], bg_map=fsaverage['sulc_left'], 
#                                       hemi='left', view='medial', bg_on_data=True)
#stat_img_surf3D = plotting.view_img_on_surf(spmT_img_FDR[2], cmap="black_red", colorbar=True, colorbar_height=0.5, 
#                                            surf_mesh='fsaverage', title=con_names[2])
stat_img_stat_map_slize = plotting.view_img(spmT_img_FDR[2], threshold=0, cut_coords=[-42, -16, 52], title=con_names[2])
stat_img_stat_map_slize.save_as_html(os.path.join(FDR_dirs[2], "spmT3_splittemplate_GM.html"))
stat_img = plotting.plot_stat_map(spmT_img_FDR[2], threshold=0, display_mode="z", cut_coords=10, title=con_names[2])
stat_img.savefig(os.path.join(FDR_dirs[2], "spmT3_Zslice10splittemplate_GM.png"))
stat_img2 = plotting.plot_stat_map(spmT_img_FDR[2], threshold=0, display_mode="z", cut_coords=10, bg_img=DARTEL_template_dir + "grouptemplate_6_GM.nii", title=con_names[2])
stat_img2.savefig(os.path.join(FDR_dirs[2], "spmT3_Zslice10_ontemplate_splittemplate_GM.png"))

Testing Zone

In [219]:
print(malefemale_fdata[100,100,:].astype(np.float32))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]


In [ ]:
# make presentation figure showing middle slice of one struct and one fMRI
# activate in-line magic
%matplotlib inline

# get the relevant volume out of the 4D nifti
fmri1=nimg.load_img(FMRI[0])
fmri1_middle=nimg.index_img(fmri1, fmri1.shape[3] // 2)

#plot everything
bettercolours=plot_epi(fmri1_middle, cut_coords=(10,10,40),
          display_mode='ortho', cmap="CMRmap", draw_cross=False, annotate=False)
normal=plot_epi(fmri1_middle, cut_coords=(10,10,40),
          display_mode='ortho', cmap="gray", draw_cross=False, annotate=False)

# safe files for presentation

bettercolours.savefig("/fMRI/fMRI_bettercolours.png")
normal.savefig("/fMRI/fMRI_normal.png")
struct.savefig("/fMRI/structMRI_normal.png")